<a href="https://colab.research.google.com/github/AnuSanal/Anu-Sanal/blob/master/CaseStudy4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

In [3]:
col_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym',  'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
feature_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym',  'fM3Long', 'fM3Trans', 'fAlpha', 'fDist']
data = pd.read_csv('magic_gamma_telescope04_ (2).csv', names=col_names)
X = data[feature_names]
Y = data['class']
data.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
1,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828,g
2,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.261,g
3,162.052,136.031,4.0612,0.0374,0.0187,116.741,-64.858,-45.216,76.96,256.788,g
4,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.449,116.737,g


In [6]:
df = data.copy()


In [8]:
def knn():
    neigh_score =[]
    for i in range(1,30,1):
        knn = KNeighborsClassifier(n_neighbors=i)
        knn.fit(X_train,Y_train)
        pred = knn.predict(X_test)
        score = accuracy_score(Y_test,pred)
        neigh_score.append((i, score))
    k = max(neigh_score,key=lambda x:x[1])[0]
    knn = KNeighborsClassifier(n_neighbors=k)
    return knn

In [9]:
def lg_reg():
    penalty_score = []
    penalty_options = ['l1', 'l2']
    for i in penalty_options:
        logstic_model = LogisticRegression(penalty=i)
        logstic_model.fit(X_train,Y_train)
        pred = logstic_model.predict(X_test)
        score = accuracy_score(Y_test,pred)
        penalty_score.append((i, score))
    pen = max(penalty_score,key=lambda x:x[1])[0]
    logstic_model = LogisticRegression(penalty=pen)
    return logstic_model

In [10]:
def rand_forest():
    params =[]
    for i in ['gini','entropy']:
        for j in range(1,100,1):
            forest = RandomForestClassifier(n_estimators=j, max_depth=2,random_state=0,criterion=i)
            forest.fit(X_train,Y_train)
            pred = forest.predict(X_test)
            score = accuracy_score(Y_test,pred)
            params.append((i,j,score))
    heighest = max(params,key=lambda x:x[2])
    forest = RandomForestClassifier(n_estimators=heighest[1], max_depth=2,random_state=0,criterion=heighest[0])
    return forest

In [11]:
def ada_boost():
    params = []
    for i in range(1,100,1):
        clf = AdaBoostClassifier(n_estimators=i, random_state=0)
        clf.fit(X_train,Y_train)
        pred = clf.predict(X_test)
        score = accuracy_score(Y_test,pred)
        params.append((i,score))

    clf = AdaBoostClassifier(n_estimators=max(params,key=lambda x:x[1])[0], random_state=0)
    return clf

In [12]:
def dec_tree():
    tree = DecisionTreeClassifier(criterion='entropy')
    return tree

In [13]:
tunning = {"Naive Bayes":GaussianNB, "KNN": knn, "Logistic Regression": lg_reg, "Random Forest": rand_forest, "Ada Boost": ada_boost, "Decision Tree":dec_tree}

In [14]:
def best_k(X):
    err = X.shape[0]
    best_val = 0
    
    for i in range(1, X.shape[1]):
    
        X_new = SelectKBest(f_classif, k=i).fit_transform(X, Y)
        model = GaussianNB()
        y_pred = model.fit(X_new, Y).predict(X_new)
        num = (Y != y_pred).sum()
        if num < err :
            err = num
            best_val = i
    
    return best_val

In [15]:
X = preprocessing.StandardScaler().fit_transform(X)
K = best_k(X)


X_new = SelectKBest(f_classif, k=K).fit_transform(X, Y)
X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y, train_size=0.7)

for name in tunning:
    model = tunning[name]()
    y_pred = model.fit(X_train, Y_train).predict(X_train)
    
    print(name)
    print("Number of mislabeled points %d out of %d total points."% ((Y_train != y_pred).sum(), X_train.shape[0]))
    r = precision_recall_fscore_support(Y_train, y_pred)
    print("precision: ",str( (r[0][0]+r[0][1])/2 ), "\t Recall: ", str( (r[1][0]+r[1][1])/2 ), "\t FScore: ", str( (r[2][0]+r[2][1])/2 ))
    print("Model accuracy =" , model.score(X_test,Y_test))   
    print("=======================================")

ValueError: ignored